# Prediksi Gangguan Tidur (Sleep Disorder)

Permasalahan yang ingin diselesaikan dalam proyek ini adalah memprediksi kualitas tidur seseorang berdasarkan faktor gaya hidup. Tidur yang berkualitas dan gaya hidup sehat memiliki peran penting dalam menjaga kesejahteraan fisik dan mental. Dengan memahami hubungan antara berbagai faktor ini, kita dapat memberikan rekomendasi yang lebih baik untuk meningkatkan kualitas tidur dan kesehatan secara keseluruhan.

Dataset yang digunakan dalam proyek ini adalah "[Sleep Health and Lifestyle Dataset](https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset/data)" yang diambil dari platform Kaggle. Dataset ini berisi 400 record dengan 13 atribut yang mencakup berbagai aspek terkait kebiasaan tidur dan gaya hidup.

Atribut dalam Dataset:

- **Demografi:**
  - Gender (Jenis Kelamin: Male/Female)
  - Age (Usia dalam tahun)
  - Occupation (Pekerjaan atau profesi)

- **Faktor Tidur:**
  - Sleep Duration (Lama tidur dalam jam per hari)
  - Quality of Sleep (Kualitas tidur, skala 1-10)
  
- **Faktor Gaya Hidup:**
  - Physical Activity Level (Tingkat aktivitas fisik, menit/hari)
  - Stress Level (Tingkat stres, skala 1-10)
  - BMI Category (Kategori BMI: Underweight, Normal, Overweight)
  
- **Indikator Kesehatan:**
  - Blood Pressure (Tekanan darah, sistolik/diastolik)
  - Heart Rate (Detak jantung, bpm)
  - Daily Steps (Jumlah langkah harian)
  
- **Gangguan Tidur:**
  - Sleep Disorder (Gangguan tidur: None, Insomnia, Sleep Apnea)

Detail tentang Kolom Sleep Disorder:
- **None:** Individu tidak memiliki gangguan tidur.
- **Insomnia:** Kesulitan untuk tidur atau tetap tidur, yang berdampak pada kualitas tidur.
- **Sleep Apnea:** Gangguan tidur akibat berhentinya pernapasan selama tidur, yang dapat menyebabkan gangguan kesehatan.


Langkah pertama yang akan dilakukan adalah dengan import library yang dibutuhkan dan membaca dataset yang akan digunakan.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2, trainer_pb2, tuner_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.components import Evaluator, Pusher
from tfx.proto import pusher_pb2
import os

## Set Variables

Selaanjutnya, kita akan menentukan variabel untuk membuat end-to-end pipeline menggunakan TFX dengan mendefinisikan beberapa konfigurasi seperti nama pipeline, lokasi dataset, lokasi metadata, dan lain-lain.

In [41]:
PIPELINE_NAME = "krisna_santosa-pipeline"
PIPELINE_ROOT = os.path.join('output', PIPELINE_NAME)
METADATA_PATH = os.path.join('output/metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = 'output/serving_model'
MODULE_DIR = "modules"
DATA_ROOT = "data"

Lakukan inisialisasi instance `InteractiveContext` untuk mengatur dan menjalankan pipeline TFX secara interaktif yang menerima parameter berupa nama pipeline dan lokasi metadata.

In [3]:
# Initialize InteractiveContext
interactive_context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT
)

## Data Ingestion

Langkah pertama dalam pipeline adalah melakukan data ingestion. Dalam kasus ini, dataset yang digunakan adalah dataset obesitas yang telah dijelaskan sebelumnya. Dataset ini akan dibaca menggunakan komponen `CsvExampleGen` yang akan menghasilkan output berupa dataset yang telah di-preprocess. Kode di bawah ini akan membagi dataset menjadi dua bagian, yaitu 80% untuk training dan 20% untuk testing.

In [4]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output
)

Untuk melihat komponen `ExampleGen` secara interaktif, kita dapat menjalankan komponen tersebut menggunakan object InteractiveContext() yang telah kita definisikan sebelumnya.

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

Setelah data di-preprocess, langkah selanjutnya adalah melakukan data validation, ada tiga komponen yang digunakan dalam data validation, yaitu `StatisticsGen`, `SchemaGen`, dan `ExampleValidator`. Komponen `StatisticsGen` akan menghasilkan statistik deskriptif dari dataset, komponen `SchemaGen` akan menghasilkan skema dari dataset, dan komponen `ExampleValidator` akan memvalidasi data berdasarkan skema yang telah dihasilkan oleh komponen `SchemaGen`.

### Summary Statistics

Komponen ini akan berisi statistik deskriptif dari dataset, seperti jumlah data, rata-rata, standar deviasi, dan lain-lain. Kode di bawah ini akan menampilkan statistik deskriptif dari dataset. Komponen ini menerima input berupa dataset yang telah di-preprocess oleh komponen `ExampleGen`.

In [6]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Data Schema

Komponen ini akan menghasilkan skema dari dataset, seperti tipe data, domain, dan lain-lain. Kode di bawah ini akan menampilkan skema dari dataset. Komponen ini menerima input berupa dataset yang telah di-preprocess oleh komponen `ExampleGen`.

In [8]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
interactive_context.show(schema_gen.outputs["schema"])


,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'blood_pressure',STRING,required,,'blood_pressure'
'bmi_category',STRING,required,,'bmi_category'
'daily_steps',INT,required,,-
'gender',STRING,required,,'gender'
'heart_rate',INT,required,,-
'occupation',STRING,required,,'occupation'
'person_id',INT,required,,-
'physical_activity_level',INT,required,,-


,Values
Domain,
'blood_pressure',"'115/75', '115/78', '117/76', '118/75', '119/77', '120/80', '122/80', '125/80', '125/82', '126/83', '128/84', '128/85', '129/84', '130/85', '131/86', '132/87', '135/90', '139/91', '140/90', '140/95', '142/92', '118/76', '121/79', '130/86', '135/88'"
'bmi_category',"'Normal', 'Normal Weight', 'Obese', 'Overweight'"
'gender',"'Female', 'Male'"
'occupation',"'Accountant', 'Doctor', 'Engineer', 'Lawyer', 'Manager', 'Nurse', 'Salesperson', 'Scientist', 'Software Engineer', 'Teacher', 'Sales Representative'"
'sleep_disorder',"'Insomnia', 'None', 'Sleep Apnea'"


### Anomalies Detection (Validator)

Pada komponen ini, kita akan melakukan validasi data berdasarkan skema yang telah dihasilkan oleh komponen `SchemaGen`. Komponen ini akan mendeteksi anomali pada dataset, seperti data yang hilang, data yang tidak sesuai dengan skema, dan lain-lain.

In [10]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(example_validator.outputs['anomalies'])


Berdasarkan hasil tersebut tidak terdapat anomali yang ditemukan dalam dataset. Aritnya data siap masuk ke tahap selanjutnya, yaitu preprocessing.

## Data Preprocessing

Setelah tahap data validation, langkah selanjutnya adalah melakukan data preprocessing. Dalam kasus ini, kita akan melakukan data preprocessing dengan menggunakan komponen `Transform`. Komponen ini akan melakukan preprocessing data, seperti normalisasi, one-hot encoding, dan lain-lain. Untuk melakukan preprocessing data, kita perlu mendefinisikan file module yang berisi fungsi preprocessing data. 

In [12]:
TRANSFORM_MODULE_FILE = os.path.join(MODULE_DIR, "sleep_disorder_transform.py")


In [45]:
%%writefile {TRANSFORM_MODULE_FILE}
"""Defines the preprocessing functions for transforming features using TensorFlow Transform."""

import tensorflow_transform as tft

NUMERIC_FEATURES = [
    'age',
    'daily_steps',
    'heart_rate',
    'physical_activity_level',
    'quality_of_sleep',
    'sleep_duration',
    'stress_level'
]

CATEGORICAL_FEATURES = [
    'bmi_category',
    'blood_pressure',
    'gender',
    'occupation'
]

LABEL_KEY = 'sleep_disorder'


def transformed_name(feature_name):
    """Append '_xf' to feature name to denote transformed feature."""
    return f'{feature_name}_xf'


def scale_numeric_features(inputs):
    """Scale numeric features to z-score."""
    return {
        transformed_name(feature): tft.scale_to_z_score(inputs[feature])
        for feature in NUMERIC_FEATURES
    }


def encode_categorical_features(inputs):
    """Convert categorical features to indices using vocabulary."""
    return {
        transformed_name(feature): tft.compute_and_apply_vocabulary(
            inputs[feature], vocab_filename=feature)
        for feature in CATEGORICAL_FEATURES
    }


def encode_label(inputs):
    """Convert label to int64 using vocabulary."""
    return {
        transformed_name(LABEL_KEY): tft.compute_and_apply_vocabulary(
            inputs[LABEL_KEY], vocab_filename=LABEL_KEY)
    }


def preprocessing_fn(inputs):
    """Preprocess input features into transformed features."""
    outputs = {}

    # Process numeric, categorical features and label
    outputs.update(scale_numeric_features(inputs))
    outputs.update(encode_categorical_features(inputs))
    outputs.update(encode_label(inputs))

    return outputs


Overwriting modules\sleep_disorder_transform.py


Setelah file module preprocessing data telah dibuat, kita dapat mendefinisikan komponen `Transform` dengan mendefinisikan fungsi preprocessing data yang telah dibuat sebelumnya. Komponen ini menerima input berupa dataset yang telah di-preprocess oleh komponen `ExampleGen` dan output berupa dataset yang telah di-preprocess.

In [14]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: output\krisna_santosa-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\364761ff699a41e18730e9ac67a290f5\assets


INFO:tensorflow:Assets written to: output\krisna_santosa-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\364761ff699a41e18730e9ac67a290f5\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Sampai tahap ini, kita telah melakukan data ingestion, data validation, dan data preprocessing. Langkah selanjutnya adalah melakukan training model menggunakan komponen `Trainer`.

## Model Development

Pada tahap ini, kita akan melakukan training model menggunakan komponen `Trainer`. Komponen ini akan melakukan training model menggunakan dataset yang telah di-preprocess oleh komponen `Transform`. Tetapi sebelum itu kita akan melakukan tuning hyperparameter menggunakan komponen `Tuner` terlebih dahulu. 

In [15]:
TUNER_MODULE_FILE = os.path.join(MODULE_DIR, "sleep_disorder_tuner.py")

### Tuner

Komponen ini akan melakukan tuning hyperparameter pada model yang akan digunakan. Kita perlu mendefinisikan file module yang berisi fungsi untuk membuat model, fungsi untuk meng-compile model, dan fungsi untuk melakukan tuning hyperparameter.

In [46]:
%%writefile {TUNER_MODULE_FILE}
""" Module for hyperparameter tuning with Keras Tuner. """

from typing import NamedTuple, Dict, Text, Any
import kerastuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers  # pylint: disable=import-error
from kerastuner.engine import base_tuner
from tfx.components.trainer.fn_args_utils import FnArgs

# Constants for feature categories and label key
NUMERIC_FEATURES = [
    'age', 'daily_steps', 'heart_rate', 'physical_activity_level',
    'quality_of_sleep', 'sleep_duration', 'stress_level'
]

CATEGORICAL_FEATURES = [
    'bmi_category', 'blood_pressure', 'gender', 'occupation'
]

LABEL_KEY = "sleep_disorder"


def transformed_name(key: str) -> str:
    """
    Generate transformed feature name by appending '_xf' to the original key.

    Args:
        key (str): The original feature name.

    Returns:
        str: The transformed feature name.
    """
    return f"{key}_xf"


# Named tuple for Tuner function result
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])


def _gzip_reader_fn(filenames: str) -> tf.data.TFRecordDataset:
    """
    Create a GZIP reader for dataset files.

    Args:
        filenames (str): Path to the dataset file.

    Returns:
        tf.data.TFRecordDataset: A GZIP reader for the given dataset file.
    """
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _input_fn(file_pattern: str,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 128) -> tf.data.Dataset:
    """
    Input function to load and preprocess the dataset using the transformed features.

    Args:
        file_pattern (str): File pattern for the dataset files.
        tf_transform_output (tft.TFTransformOutput): TensorFlow Transform output.
        batch_size (int): The batch size for dataset loading.

    Returns:
        tf.data.Dataset: The dataset with transformed features and labels.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=1,
        shuffle=True,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset


def create_numeric_input_layer(feature_name: str) -> layers.Layer:
    """
    Helper function to create the input layer for numeric features.

    Args:
        feature_name (str): The name of the numeric feature.

    Returns:
        layers.Layer: The input layer for the numeric feature.
    """
    input_layer = layers.Input(shape=(1,), name=transformed_name(feature_name))
    norm_layer = layers.BatchNormalization()(input_layer)
    return norm_layer


def create_categorical_input_layer(feature_name: str,
                                   vocab_size: int,
                                   embedding_dim: int) -> layers.Layer:
    """
    Helper function to create the input layer for categorical features with embedding.

    Args:
        feature_name (str): The name of the categorical feature.
        vocab_size (int): The vocabulary size for the feature.
        embedding_dim (int): The embedding dimension.

    Returns:
        layers.Layer: The input layer for the categorical feature.
    """
    input_layer = layers.Input(
        shape=(1,), name=transformed_name(feature_name), dtype=tf.int64)
    safe_input = tf.where(
        tf.logical_or(input_layer < 0, input_layer >= vocab_size),
        tf.zeros_like(input_layer),
        input_layer
    )
    embedding = layers.Embedding(
        vocab_size, embedding_dim, mask_zero=True, name=f'embedding_{feature_name}')(safe_input)
    return layers.Flatten()(embedding)


def model_builder(hp, tf_transform_output: tft.TFTransformOutput) -> tf.keras.Model:
    """
    Build a Keras model for sleep disorder prediction with hyperparameter tuning support.

    Args:
        hp: The hyperparameter object used for tuning.
        tf_transform_output (tft.TFTransformOutput): TensorFlow Transform output.

    Returns:
        tf.keras.Model: The compiled Keras model.
    """
    inputs = []
    encoded_features = []

    # Process numeric features
    for feature_name in NUMERIC_FEATURES:
        encoded_features.append(create_numeric_input_layer(feature_name))

    # Process categorical features
    embedding_dim = hp.Int('embedding_dim', 8, 32, step=8)
    for feature_name in CATEGORICAL_FEATURES:
        vocab_size = tf_transform_output.vocabulary_size_by_name(feature_name)
        encoded_features.append(create_categorical_input_layer(
            feature_name, vocab_size, embedding_dim))

    # Concatenate encoded features
    concat_features = layers.concatenate(encoded_features)

    # Add hidden layers with hyperparameter tuning
    num_hidden_layers = hp.Int('num_hidden_layers', 2, 4)
    for i in range(num_hidden_layers):
        units = hp.Int(f'units_{i}', 32, 256, step=32)
        dropout_rate = hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)
        concat_features = layers.Dense(
            units, activation='relu')(concat_features)
        concat_features = layers.BatchNormalization()(concat_features)
        concat_features = layers.Dropout(dropout_rate)(concat_features)

    # Output layer
    outputs = layers.Dense(3, activation='softmax')(concat_features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile model with learning rate tuning
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """
    Create a hyperparameter tuning function for the sleep disorder prediction model.

    Args:
        fn_args (FnArgs): Arguments passed to the tuning function.

    Returns:
        TunerFnResult: Named tuple containing the tuner and fitting arguments.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Hyperparameter tuner setup
    tuner = kt.Hyperband(
        hypermodel=lambda hp: model_builder(hp, tf_transform_output),
        objective='val_accuracy',
        max_epochs=30,
        factor=3,
        directory=fn_args.working_dir,
        project_name='sleep_disorder_tuning',
        overwrite=True
    )

    # Prepare train and eval datasets
    train_dataset = _input_fn(fn_args.train_files, tf_transform_output)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output)

    # Early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_dataset,
            'validation_data': eval_dataset,
            'callbacks': [early_stopping]
        }
    )


Overwriting modules\sleep_disorder_tuner.py


Setelah file module tuning hyperparameter telah dibuat, kita dapat mendefinisikan komponen `Tuner` dengan mendefinisikan fungsi tuning hyperparameter yang telah dibuat sebelumnya. Komponen ini menerima input berupa dataset yang telah di-preprocess oleh komponen `Transform`.

In [32]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Trial 90 Complete [00h 00m 02s]
val_accuracy: 0.8795180916786194

Best val_accuracy So Far: 0.9036144614219666
Total elapsed time: 00h 03m 02s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in output\krisna_santosa-pipeline\.temp\13\sleep_disorder_tuning
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 32
num_hidden_layers: 3
units_0: 128
dropout_0: 0.30000000000000004
units_1: 128
dropout_1: 0.2
learning_rate: 0.00028965674098173344
units_2: 64
dropout_2: 0.1
units_3: 64
dropout_3: 0.30000000000000004
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0076
Score: 0.9036144614219666
Trial summary
Hyperparameters:
embedding_dim: 24
num_hidden_layers: 4
units_0: 224
dropout_0: 0.2
units_1: 96
dropout_1: 0.30000000000000004
learning_rate: 0.0025937845981801036
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 3
tuner/round: 2
units_2: 32
dropout_2: 0.1
units_3: 32
dropout_3: 0.1
tuner/trial_id: 0045
Score: 0.891566276550293
Trial summary
Hyperparameters:
embedding_dim: 24
num_hidden_layers: 4
units_0: 224
dropout_0: 0.2
units_1: 96
dropout_1: 0.30000000000000004
learning_rate: 0.0025937845

ExecutionResult(
    component_id: Tuner
    execution_id: 13
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Model Training

Setelah tuning hyperparameter selesai, kita dapat melakukan training model menggunakan komponen `Trainer`. Komponen ini akan melakukan training model menggunakan dataset yang telah di-preprocess oleh komponen `Transform` dan hyperparameter yang telah di-tuning oleh komponen `Tuner`. Kita akan definisikan file module yang berisi fungsi untuk membuat model, fungsi untuk meng-compile model, dan fungsi untuk melakukan training model.

In [43]:
TRAINER_MODULE_FILE = os.path.join(MODULE_DIR,"sleep_disorder_trainer.py")

In [47]:
%%writefile {TRAINER_MODULE_FILE}
""" Module to train a Keras model with TensorFlow Transform preprocessing. """

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers  # pylint: disable=import-error
from tfx.components.trainer.fn_args_utils import FnArgs

NUMERIC_FEATURES = [
    'age', 'daily_steps', 'heart_rate', 'physical_activity_level',
    'quality_of_sleep', 'sleep_duration', 'stress_level'
]

CATEGORICAL_FEATURES = [
    'bmi_category', 'blood_pressure', 'gender', 'occupation'
]

LABEL_KEY = "sleep_disorder"


def transformed_name(key: str) -> str:
    """
    Transforms the feature name by appending '_xf' to the original key.

    Args:
        key (str): The original feature name.

    Returns:
        str: The transformed feature name.
    """
    return f"{key}_xf"


def gzip_reader_fn(filenames: str) -> tf.data.TFRecordDataset:
    """
    Reads a GZIP file into a TensorFlow TFRecord dataset.

    Args:
        filenames (str): The file path(s) to read from.

    Returns:
        tf.data.TFRecordDataset: The dataset containing the records.
    """
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern: str,
             tf_transform_output: tft.TFTransformOutput,
             batch_size: int = 128) -> tf.data.Dataset:
    """
    Creates an input function that loads and preprocesses the dataset.

    Args:
        file_pattern (str): The file path(s) to load.
        tf_transform_output (tft.TFTransformOutput): TensorFlow Transform output.
        batch_size (int): The batch size for data loading.

    Returns:
        tf.data.Dataset: The dataset ready for training or evaluation.
    """
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=None,
        shuffle=True,
        shuffle_buffer_size=1000,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset


def _get_serve_tf_examples_fn(model: tf.keras.Model,
                              tf_transform_output: tft.TFTransformOutput):
    """
    Returns a serving function for the TensorFlow model.

    Args:
        model (tf.keras.Model): The trained TensorFlow model.
        tf_transform_output (tft.TFTransformOutput): TensorFlow Transform output.

    Returns:
        function: The serving function to handle incoming requests.
    """
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples: tf.Tensor):
        """
        Process and transform the input examples for serving.

        Args:
            serialized_tf_examples (tf.Tensor): Serialized TF examples to process.

        Returns:
            tf.Tensor: Model predictions.
        """
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs):
    """
    Main function to train the model with the provided arguments.

    Args:
        fn_args (FnArgs): The arguments passed to the trainer function.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Get dataset and compute steps per epoch
    batch_size = 128
    train_dataset = input_fn(
        fn_args.train_files, tf_transform_output, batch_size)
    eval_dataset = input_fn(
        fn_args.eval_files, tf_transform_output, batch_size)

    # Load best hyperparameters
    hp = fn_args.hyperparameters['values']

    # Build model with best hyperparameters
    model = build_model(hp, tf_transform_output)

    steps_per_epoch = 26
    validation_steps = 7

    # Train model
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=30,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=[tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True
        )]
    )

    # Save model with serving signature
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'))
    }
    model.save(fn_args.serving_model_dir,
               save_format='tf', signatures=signatures)


def build_model(hp: dict, tf_transform_output: tft.TFTransformOutput) -> tf.keras.Model:
    """
    Build and compile a Keras model based on hyperparameters.

    Args:
        hp (dict): The dictionary of hyperparameters.
        tf_transform_output (tft.TFTransformOutput): TensorFlow Transform output.

    Returns:
        tf.keras.Model: The compiled Keras model.
    """
    # Create feature processing layers
    numeric_layers = _build_numeric_features(
        NUMERIC_FEATURES)
    categorical_layers = _build_categorical_features(
        CATEGORICAL_FEATURES, hp, tf_transform_output)

    # Concatenate all features
    concat_features = layers.concatenate(numeric_layers + categorical_layers)

    # Add hidden layers dynamically from tuner
    hidden_layers = _build_hidden_layers(concat_features, hp)

    # Output layer
    outputs = layers.Dense(7, activation='softmax')(hidden_layers)

    model = tf.keras.Model(inputs=numeric_layers +
                           categorical_layers, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def _build_numeric_features(numeric_feature_names):
    """
    Builds and processes numeric features for the model.

    Args:
        numeric_feature_names (list): The list of numeric feature names.

    Returns:
        list: A list of layers processing the numeric features.
    """
    numeric_layers = []
    for feature_name in numeric_feature_names:
        input_layer = layers.Input(
            shape=(1,), name=transformed_name(feature_name))
        norm_layer = layers.BatchNormalization()(input_layer)
        numeric_layers.append(norm_layer)
    return numeric_layers


def _build_categorical_features(categorical_feature_names, hp, tf_transform_output):
    """
    Builds and processes categorical features with embeddings for the model.

    Args:
        categorical_feature_names (list): The list of categorical feature names.
        hp (dict): Hyperparameters for the model.
        tf_transform_output: The transform output containing preprocessing details.

    Returns:
        list: A list of layers processing the categorical features.
    """
    categorical_layers = []
    for feature_name in categorical_feature_names:
        vocab_size = tf_transform_output.vocabulary_size_by_name(feature_name)
        input_layer = layers.Input(
            shape=(1,), name=transformed_name(feature_name), dtype=tf.int64)

        safe_input = tf.where(
            tf.logical_or(input_layer < 0, input_layer >= vocab_size),
            tf.zeros_like(input_layer),
            input_layer
        )

        embedding = layers.Embedding(
            vocab_size,
            hp.get('embedding_dim'),
            mask_zero=True
        )(safe_input)

        embedding_flat = layers.Flatten()(embedding)
        categorical_layers.append(embedding_flat)

    return categorical_layers


def _build_hidden_layers(concat_features, hp):
    """
    Adds hidden layers to the model based on the hyperparameters.

    Args:
        concat_features (tensor): The concatenated feature layer.
        hp (dict): Hyperparameters for the model.

    Returns:
        tensor: The layer after applying all hidden layers and dropout.
    """
    hidden_layer = concat_features
    for i in range(hp.get('num_hidden_layers')):
        units = hp.get(f'units_{i}')
        dropout_rate = hp.get(f'dropout_{i}')
        hidden_layer = layers.Dense(units, activation='relu')(hidden_layer)
        hidden_layer = layers.BatchNormalization()(hidden_layer)
        hidden_layer = layers.Dropout(dropout_rate)(hidden_layer)

    return hidden_layer


Overwriting modules\sleep_disorder_trainer.py


In [35]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

Epoch 1/30
26/26 [==============================] - 1s 11ms/step - loss: 1.7618 - accuracy: 0.3954 - val_loss: 1.7408 - val_accuracy: 0.6875
Epoch 2/30
26/26 [==============================] - 0s 4ms/step - loss: 1.0154 - accuracy: 0.7440 - val_loss: 1.6300 - val_accuracy: 0.4844
Epoch 3/30
26/26 [==============================] - 0s 4ms/step - loss: 0.7813 - accuracy: 0.8419 - val_loss: 1.5123 - val_accuracy: 0.5670
Epoch 4/30
26/26 [==============================] - 0s 4ms/step - loss: 0.6724 - accuracy: 0.8768 - val_loss: 1.3734 - val_accuracy: 0.6484
Epoch 5/30
26/26 [==============================] - 0s 4ms/step - loss: 0.6063 - accuracy: 0.8933 - val_loss: 1.2269 - val_accuracy: 0.7812
Epoch 6/30
26/26 [==============================] - 0s 4ms/step - loss: 0.5385 - accuracy: 0.9108 - val_loss: 1.0857 - val_accuracy: 0.8080
Epoch 7/30
26/26 [==============================] - 0s 4ms/step - loss: 0.5312 - accuracy: 0.9159 - val_loss: 0.9565 - val_accuracy: 0.8192
Epoch 8/30
26/26 [=

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: output\krisna_santosa-pipeline\Trainer\model\14\Format-Serving\assets


INFO:tensorflow:Assets written to: output\krisna_santosa-pipeline\Trainer\model\14\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Model Analysis and Validation

Setelah training model selesai, langkah selanjutnya adalah melakukan analisis model dan validasi model. Dalam kasus ini, kita akan menggunakan komponen `Resolver` dan `Evaluator`. Resolver berperan untuk menentukan baseline model yang akan digunakan untuk membandingkan model yang telah di-training. Sedangkan Evaluator berperan untuk mengevaluasi model yang telah di-training.

### Resolver Component

Pada komponen ini, kita akan menentukan baseline model yang akan digunakan untuk membandingkan model yang telah di-training.

In [36]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 15
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator Component

Pada komponen ini, kita akan mengevaluasi model yang telah di-training. Komponen ini akan menghasilkan beberapa metric evaluasi model, seperti accuracy, precision, recall, dan lain-lain. Kode di bawah ini akan menampilkan metric evaluasi model dengan threshold 0.85.

In [37]:
LABEL_KEY = "sleep_disorder"

def transformed_name(key):
    return key + '_xf'

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(
            label_key=transformed_name(LABEL_KEY)
        )],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='SparseCategoricalAccuracy'),
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.85}
                    ),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': 0.0001}
                    )
                )
            )
        ])
    ]
)

Setelah membuat konfigurasi untuk komponen `Evaluator`, kita dapat mengevaluasi model yang telah di-training dengan menjalankan komponen `Evaluator` pada kode di bawah ini.

In [38]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Untuk dapat melihat hasil evaluasi model dengan visualisasi, kita menggunakan  `tfma.view.render_slicing_metrics` yang akan menampilkan metric evaluasi model dengan visualisasi.

In [39]:
# Visualize evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

## Pusher

Setelah model dievaluasi, langkah terakhir adalah melakukan push model ke production. Pada kasus ini, kita akan menggunakan komponen `Pusher` untuk melakukan push model ke production. Komponen ini akan melakukan menyimpan model yang telah di-training ke storage yang telah ditentukan.

In [42]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 18
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))